<a href="https://colab.research.google.com/github/yujinc129-oss/Drama_data_project/blob/main/notebooks/03_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
%pip install PyGithub

KeyboardInterrupt: 

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
from google.colab import userdata
from github import Github, GithubException


In [22]:
drama =pd.read_json('https://github.com/yujinc129-oss/Drama_data_project/raw/refs/heads/main/data/processed/02_pre_2', orient="records")

In [23]:
drama

,actor,drama,role,gender,genres,day,network,age,start airing,score
0,junghaein,snowdrop,Main Role,Male,"[drama, society, romance]","[saturday, sunday]",[JTBC],34,2021,8.20
1,junghaein,d.p.,Main Role,Male,"[drama, action, hist_war]",[friday],[NETFLIX],34,2021,8.43
2,junghaein,apieceofyourmind,Main Role,Male,"[drama, romance]","[monday, tuesday]",[TVN],33,2020,7.97
3,junghaein,onespringnight,Main Role,Male,"[drama, romance]","[thursday, wednesday]","[MBC, NETFLIX]",32,2019,8.17
4,junghaein,somethingintherain,Main Role,Male,"[drama, romance]","[friday, saturday]","[NETFLIX, JTBC]",31,2018,7.92
...,...,...,...,...,...,...,...,...,...,...
2172,kimjongkook,theproducers,Support Role,Male,"[drama, comedy, romance]","[friday, saturday]",[KBS],40,2015,7.64
2173,jungyoonhak,myunfortunateboyfriend,Main Role,Male,"[drama, romance]","[friday, saturday]",[MBC],32,2015,7.54
2174,jungjoonyoung,thelover,Main Role,Male,"[drama, comedy, romance]",[thursday],[CJ],27,2015,7.80
2175,kimjoon,cityofthesun,Main Role,Male,"[drama, thriller, romance, society]","[friday, saturday]",[MBC],32,2015,7.70


In [24]:
#주연비율 파생변수 생성
drama_cnt = drama.actor.value_counts(sort=False)
main_cnt =drama[drama['role']=='Main Role'].groupby('actor',sort=False,)['drama'].count()
main_cnt = main_cnt.fillna(0)
drama_cnt = drama_cnt.fillna(0)
ratio = main_cnt / drama_cnt
ratio = ratio.replace([np.inf, -np.inf], np.nan).fillna(0)

ratio =ratio.reset_index()

drama = pd.merge(drama, ratio, how='left')

drama.rename(columns={0:'main_ratio'},inplace=True)

drama

,actor,drama,role,gender,genres,day,network,age,start airing,score,main_ratio
0,junghaein,snowdrop,Main Role,Male,"[drama, society, romance]","[saturday, sunday]",[JTBC],34,2021,8.20,0.75
1,junghaein,d.p.,Main Role,Male,"[drama, action, hist_war]",[friday],[NETFLIX],34,2021,8.43,0.75
2,junghaein,apieceofyourmind,Main Role,Male,"[drama, romance]","[monday, tuesday]",[TVN],33,2020,7.97,0.75
3,junghaein,onespringnight,Main Role,Male,"[drama, romance]","[thursday, wednesday]","[MBC, NETFLIX]",32,2019,8.17,0.75
4,junghaein,somethingintherain,Main Role,Male,"[drama, romance]","[friday, saturday]","[NETFLIX, JTBC]",31,2018,7.92,0.75
...,...,...,...,...,...,...,...,...,...,...,...
2172,kimjongkook,theproducers,Support Role,Male,"[drama, comedy, romance]","[friday, saturday]",[KBS],40,2015,7.64,0.00
2173,jungyoonhak,myunfortunateboyfriend,Main Role,Male,"[drama, romance]","[friday, saturday]",[MBC],32,2015,7.54,1.00
2174,jungjoonyoung,thelover,Main Role,Male,"[drama, comedy, romance]",[thursday],[CJ],27,2015,7.80,1.00
2175,kimjoon,cityofthesun,Main Role,Male,"[drama, thriller, romance, society]","[friday, saturday]",[MBC],32,2015,7.70,1.00


In [25]:
#플랫폼 등급 컬럼 추가
# 1. 등급 매핑 딕셔너리 정의 (숫자가 작을수록 높은 등급)
tier_map = {'NETFLIX':1, 'TVN':1, 'JTBC':1, 'KBS':2, 'MBC':2, 'SBS':2, 'OCN':2, 'ETC':3}
default_tier = 3

# 2. 가장 높은 등급을 찾아주는 함수 정의
def get_highest_tier(platform_list):
    # 리스트에 있는 각 플랫폼의 등급을 찾음
    # tier_map에 없는 플랫폼은 default_tier로 처리
    tiers = [tier_map.get(p, default_tier) for p in platform_list]

    # 등급 리스트가 비어있으면 기본 등급 반환
    if not tiers:
        return default_tier

    # 가장 작은 숫자(가장 높은 등급)를 반환
    return min(tiers)


# 3. apply 함수를 사용하여 'platform_tier' 컬럼 생성
drama['platform_tier'] = drama['network'].apply(get_highest_tier)

drama

,actor,drama,role,gender,genres,day,network,age,start airing,score,main_ratio,platform_tier
0,junghaein,snowdrop,Main Role,Male,"[drama, society, romance]","[saturday, sunday]",[JTBC],34,2021,8.20,0.75,1
1,junghaein,d.p.,Main Role,Male,"[drama, action, hist_war]",[friday],[NETFLIX],34,2021,8.43,0.75,1
2,junghaein,apieceofyourmind,Main Role,Male,"[drama, romance]","[monday, tuesday]",[TVN],33,2020,7.97,0.75,1
3,junghaein,onespringnight,Main Role,Male,"[drama, romance]","[thursday, wednesday]","[MBC, NETFLIX]",32,2019,8.17,0.75,1
4,junghaein,somethingintherain,Main Role,Male,"[drama, romance]","[friday, saturday]","[NETFLIX, JTBC]",31,2018,7.92,0.75,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2172,kimjongkook,theproducers,Support Role,Male,"[drama, comedy, romance]","[friday, saturday]",[KBS],40,2015,7.64,0.00,2
2173,jungyoonhak,myunfortunateboyfriend,Main Role,Male,"[drama, romance]","[friday, saturday]",[MBC],32,2015,7.54,1.00,2
2174,jungjoonyoung,thelover,Main Role,Male,"[drama, comedy, romance]",[thursday],[CJ],27,2015,7.80,1.00,3
2175,kimjoon,cityofthesun,Main Role,Male,"[drama, thriller, romance, society]","[friday, saturday]",[MBC],32,2015,7.70,1.00,2


In [26]:
#분기 컬럼 추가
season = pd.read_csv('https://github.com/yujinc129-oss/Drama_data_project/raw/refs/heads/main/data/processed/season')
season.drop(columns='Unnamed: 0', inplace=True)
drama = pd.merge(drama, season, left_on='drama',right_on='drama_name')
drama.drop(columns='drama_name',inplace=True)

In [34]:
#drama.to_json('03_feature',orient='records')
# !wget -O git_push.py "https://github.com/yujinc129-oss/Drama_data_project/raw/refs/heads/main/utils/git_push.py"

# from git_push import git_push

# git_push('/content/03_feature')

--2025-08-06 05:35:57--  https://github.com/yujinc129-oss/Drama_data_project/raw/refs/heads/main/utils/git_push.py
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/yujinc129-oss/Drama_data_project/refs/heads/main/utils/git_push.py [following]
--2025-08-06 05:35:58--  https://raw.githubusercontent.com/yujinc129-oss/Drama_data_project/refs/heads/main/utils/git_push.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2582 (2.5K) [text/plain]
Saving to: ‘git_push.py’

git_push.py         100%[===================>]   2.52K  --.-KB/s    in 0s      

2025-08-06 05:35:58 (35.5 MB/s) - ‘git_push.py’ save

In [30]:
#감독 작가 파생변수 생성 -ing
# director = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_set/DRAMA_PROJECT/etc/drama/korean_drama.csv')
# director[['drama_name','director','screenwriter']]